In [3]:
import networkx as nx
from networkx.algorithms.isomorphism import DiGraphMatcher, GraphMatcher

## Interfacing Sage and NetworkX

In [5]:
def convert_sage_edges_to_gt(edge_list):
    """
    Accepts list of edges edge_list in the format given by sage
    Returns list of edges in the format required by graph-tool
    """
    
    return [(str(e[0]), str(e[1])) for e in edge_list]

def get_nx_bruhat_interval(W, x, y):
    bg = W.bruhat_graph(x, y)
    edges = convert_sage_edges_to_gt(bg.edges())
    G = nx.DiGraph()
    G.add_edges_from(edges)
    return G

## Example graphs

In [8]:
def k_crown(k):
    """
    Parameters:
        k : Nat
    
    Returns the k-crown graph, with vertices labelled from "x(0)" to "x(2k+1)".
    """

    edges = []
    # Edges from bottom vertex to row 1
    for i in range(1, k+1):
        edges.append(("x(0)", f"x({i})"))
    # Row 1 to row 2
    ## Margin
    edges += [("x(1)", f"x({k+1})"), ("x(1)", f"x({k+2})")]
    edges += [(f"x({k})", f"x({2*k})"), (f"x({k})", f"x({2*k-1})")]
    ## Middle
    for i in range(2, k):
        edges += [(f"x({i})", f"x({k+i-1})"), (f"x({i})", f"x({k+i+1})")]
    # Row 2 to row 3
    for i in range(1, k+1):
        edges.append((f"x({k+i})", f"x({2*k+1})"))
    out = nx.DiGraph()
    out.add_edges_from(edges)
    return out

## Operations on Graphs

### Up- and downsets of vertices in directed graphs

In [ ]:
from networkx.algorithms.traversal.breadth_first_search import bfs_tree

def up(G, x):
    return list(bfs_tree(G, x))

def down(G, x):
    return list(bfs_tree(G.reverse(), x))

### $k$-diamond closures

In [ ]:
def diamond_closure(G, X, verbose=False):
    """
    Arguments:
        G : DiGraph
        X : full subgraph of G (encoded by list of vertices)
    Compute the diamond closure of G (the smallest full subgraph of G which contains X and is diamond closed)
    """
    
    all_verts = X
    # nodes added in the last iteration
    last_verts = all_verts
    
    # Iterate until stable: 
    while True:
        if verbose:
            print("-----")
        new_verts = []
        # Go through all diamonds d of G containing at least one node from last_verts
        for d in list_diamonds(G):
            if len([x for x in d if x in last_verts]) == 0:
                continue
            # Check if three nodes of d are contained in all_verts (and one isn't)
            if len([x for x in d if x in all_verts]) != 3:
                continue
            if verbose:
                print(d)
            # Add the remaining node to new_verts
            new_node = [x for x in d if x not in all_verts][0]
            if new_node not in new_verts:
                new_verts.append(new_node)
        # Add the vertices picked up in this iteration to all_verts
        all_verts += new_verts
        last_verts = new_verts
        # If we didn't add any new nodes, then all_verts is already diamond closed.
        if new_verts == []:
            break
        if verbose:
            print("new vertices: ", new_verts)
    return sorted(all_verts, key=len)

In [ ]:
def k_diamond_closure(G, seed, k=2):
    """
    Parameters:
        G : nx.DiGraph
        seed -- List of seed vertices
        k : minimal admissible index of overdetermination

    Computes the k-diamond closure and returns a list of its vertices
    """

    known = seed
    new_vertices = ["foo"]
    all_new = []
    while len(new_vertices) > 0:
        new_vertices = []
        # Add all nodes which are overdetermined from current knowledge
        for x in G.nodes():
            if is_overdetermined(x, known, G)[0]:
                new_vertices.append(x)
        known += new_vertices
        all_new += new_vertices
    
    return known, all_new

## Graph Statistics and Checks

In [ ]:
def list_diamonds(G):
    """
    Accepts a directed graph G
    Returns a list of vertex sets of diamonds in G
    """
    
    # Create the template graph
    template = nx.DiGraph()
    template.add_edges_from([(0, 1), (0, 2), (1, 3), (2, 3)])

    # Initialize the matcher
    matcher = DiGraphMatcher(G, template)

    # Find all subgraphs isomorphic to the template
    subgraphs_with_redundancy = [mapping for mapping in matcher.subgraph_isomorphisms_iter()]

    subgraphs = []
    for s in subgraphs_with_redundancy:
        vertex_set = set(s.keys())
        if vertex_set in subgraphs:
            continue
        subgraphs.append(vertex_set)
    
    return subgraphs

In [11]:
# Helper function, ignore
def reverse_dict(d):
    """
    Reverse lookup a dictionary.
    No checks about double values are implemented (dangerous move)
    """
    d_out = dict()
    for key in d.keys():
        d_out[d[key]] = key
    return d_out 

# Main function
def sits_inside_k_crown(G, T, phi, k):
    """
    Parameters:
        G : DiGraph
        T : DiGraph -- some template graph
        phi : mapping from T to G
        k : crown index

    Checks if the image of phi is contained in some k-crown in G.
    """
    ### COMPATIBILITY ###
    # Remove edge directions from G, T
    G = nx.Graph(G)
    T = nx.Graph(T)
    # Turn phi into map
    phi_map = reverse_dict(phi)

    # List of embeddings psi: k-crown --> G
    crown = k_crown(k)
    crown = nx.Graph(crown)
    psi_matcher = GraphMatcher(G, crown)
    #print("psi_matcher : ", list(psi_matcher.subgraph_isomorphisms_iter()))

    # List of embeddings phi_bar: T --> k_crown
    phi_bar_matcher = GraphMatcher(crown, T)
    #print("phi_bar_matcher : ", list(phi_bar_matcher.subgraph_isomorphisms_iter()))

    some_commutes = False
    for psi in psi_matcher.subgraph_isomorphisms_iter():
        for phi_bar in phi_bar_matcher.subgraph_isomorphisms_iter():
            # Check if phi = psi \circ phi_bar
            # raise NotImplementedError("Finn müde finn schlafen")
            # Turn the matchers into maps
            psi_map = reverse_dict(psi)
            phi_bar_map = reverse_dict(phi_bar)

            commutes = all([psi_map[phi_bar_map[t]] == phi_map[t] for t in T.nodes()])
            if commutes:
                some_commutes = True
                break
    
    return some_commutes

In [12]:
def is_overdetermined(x, known, G, k=2):
    """
    Parameters: 
        x : Vertex of graph G
        known : Subset of "known" vertices
        G : DiGraph
        k : Nat - minimal degree of overdetermination

    Checks if a vertex x lies on k or more diamonds in a common crown
    """

    if k > 2:
        raise NotImplementedError("k-closure criterion for k >= 3 not yet implemented.")
    
    # Blueprint graph: two diamonds touching in an edge.
    template = nx.DiGraph()
    template.add_edges_from([(0, 1), (0, 2), (1, 3), (2, 3), (2, 4), (3, 5), (4, 5)])
    template = nx.Graph(template)

    # Helper function: Determine if an embedding of template into G
    #  has enough (i.e. five) known vertices
    template_is_known = lambda phi : len([x for x in phi.keys() if x in known]) == 5 

    # Initialize the matcher
    matcher = GraphMatcher(G, template)

    # Find all subgraphs isomorphic to the template (with redundancy)
    subgraphs = list(matcher.subgraph_isomorphisms_iter())
    # Require that x is mapped to 3 under mapping to eliminate one choice of automorphism
    subgraphs = [g for g in subgraphs if reverse_dict(g)[3] == x]
    # Require that the images of all vertices but 3 are known
    subgraphs = [g for g in subgraphs if template_is_known(g)]
    # Check if one of the subgraphs sits inside of some k-crown in G.
    #  The maximum crown index in G is less than or equal to the maximum degree of any vertex of G.
    max_crown_idx = max(dict(G.degree()).values())
    subgraphs = [g for g in subgraphs if any([sits_inside_k_crown(G, template, g, k) for k in range(3, max_crown_idx+1)])]
    # If at least one such mapping exists, then x is overdetermined.
    return len(subgraphs) > 0, subgraphs

In [14]:
def is_diamond_closed(G, X):
    return len(X) == len(diamond_closure(G, X))